Aprile, Amezquita, Schaber <br>CPE-695 Final Project <br>Facial Expression Detection with Limited Features

# <font color=navy> **Mask Emotion Detection Pipeline**
This code combines the prior efforts of Dlib (facial feature detection) and the model building (facial emotion detection). Given an image, it saves the image annotated with facial bounding boxes and the emotion, as predicted by the CNN, whose architecture and weights have been loaded.

In [ ]:
# Imports
from imutils import face_utils
from google.colab import drive
import numpy as np
import dlib
import cv2
import glob
import os
import matplotlib.pyplot as plt
from PIL import Image
from keras.models import model_from_json

In [ ]:
# Mount the drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Load CNN for prediction
json_file = open('/content/drive/My Drive/CPE-695 Final Project/Data/CNN_model_architecture.json', 'r')
model_json = json_file.read()
json_file.close()
model = model_from_json(model_json)

# Load weights
model.load_weights('/content/drive/My Drive/CPE-695 Final Project/Data/CNN_model_weights.h5')

# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# Prediction function
def predict_emotion(face, model):
  # Define emotions
  emotions = {
    0 : 'Angry',
    1: 'Disgust',
    2: 'Fear',
    3: 'Happy',
    4: 'Sad',
    5: 'Surprise',
    6: 'Neutral'
  }

  # Build and normalize input tensor
  X = face.reshape(1, 29, 48, 1).astype('float32') / 255

  # Predict
  pred = model.predict(X)

  # Return emotion
  return emotions[np.argmax(pred, axis=1)[0]]

In [ ]:
# Label image
def label_image(image, path):
  # Convert to grayscale
  gray = cv2.cvtColor(image.copy(), cv2.COLOR_BGR2GRAY)

  # Load predictor
  predictor = dlib.shape_predictor("/content/drive/My Drive/CPE-695 Final Project/Data/shape_predictor_68_face_landmarks.dat")
  detector = dlib.get_frontal_face_detector()

  # Get facial bounding boxes
  rects = detector(gray, 1)

  # Crop to custom images
  faces = []

  for rect in rects:
    x1 = rect.left()
    y1 = rect.top()
    x2 = rect.right()
    y2 = rect.bottom()

    faces.append(cv2.resize(gray[y1:y2, x1:x2], (48,48)))

  for f in range(len(faces)):
    img = faces[f]

    # Define new bounding box
    # Single faces are bounded, use top left and bottom right points
    bb = dlib.rectangle(0,0, img.shape[0], img.shape[1])

    # Further define box points
    x1 = bb.left()
    y1 = bb.top()
    x2 = bb.right() 
    y2 = bb.bottom() 

    # Get y-coordinate of nose (point 31)
    nose_y = predictor(image=img, box=bb).part(30).y

    # Crop image in faces
    cropped = img[0:nose_y, :]

    # Reshape (for input in model)
    faces[f] = cv2.resize(cropped, (29, 48))

  # Make predictions and draw on image
  for ind in range(len(faces)):
    # Predict emotion
    emotion = predict_emotion(faces[ind], model)

    # Draw face bounding box
    (x, y, w, h) = face_utils.rect_to_bb(rects[ind])
    cv2.rectangle(image, (x,y), (x+w, y+h), (255,0,0), 3)
    cv2.putText(image, emotion, (x-10,y-10), cv2.FONT_HERSHEY_DUPLEX, 1.5, (255,0,0), 2)
    
    # Save image
    cv2.imwrite(path, image)

### **<font color=maroon>Input images here!**

In [ ]:
# Annotate six test images
tests = glob.glob('/content/drive/My Drive/CPE-695 Final Project/Test Images/*.jpg')
save_path = '/content/drive/My Drive/CPE-695 Final Project/Test Images/annotated_'

for t in tests:
  image = cv2.imread(t)
  label_image(image, save_path + t.split('/')[-1])